In [1]:
import torchvision as thv
import torch as th
import time
from torchinfo import summary
th.manual_seed(32)
th.cuda.manual_seed(32)
th.cuda.manual_seed_all(32)

In [2]:
DEVICE = "cuda" if th.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [3]:
TRAIN_DATASET_LOC = r"E:\PyTorch\Working Data\Input Data\Elephants\dataset\train"
TEST_DATASET_LOC = r"E:\PyTorch\Working Data\Input Data\Elephants\dataset\test"

In [4]:
from torchvision.transforms import v2

train_dataset = thv.datasets.ImageFolder(root=TRAIN_DATASET_LOC,
                                        transform=v2.Compose([v2.ToImage(),
                                                              v2.Resize(size=[224,224],antialias=True),
                                                              v2.ToDtype(th.float32,scale=True)]))

test_dataset = thv.datasets.ImageFolder(root=TEST_DATASET_LOC,
                                        transform=v2.Compose([v2.ToImage(),
                                                              v2.Resize(size=[224,224],antialias=True),
                                                              v2.ToDtype(th.float32,scale=True)]))

In [5]:
train_data_loader = th.utils.data.DataLoader(train_dataset,
                                             shuffle=True,
                                             batch_size=32)

test_data_loader = th.utils.data.DataLoader(test_dataset,
                                            shuffle=True,
                                            batch_size=32)

In [6]:
train_dataset[0][0]

Image([[[0.3098, 0.2314, 0.2902,  ..., 0.4980, 0.4784, 0.4784],
        [0.3490, 0.3137, 0.3765,  ..., 0.5176, 0.5137, 0.4980],
        [0.4196, 0.3451, 0.4314,  ..., 0.5333, 0.5373, 0.5059],
        ...,
        [0.6000, 0.4196, 0.2431,  ..., 0.1686, 0.1725, 0.1882],
        [0.6275, 0.3451, 0.2392,  ..., 0.1529, 0.1686, 0.2078],
        [0.6275, 0.3451, 0.2392,  ..., 0.1569, 0.1686, 0.2078]],

       [[0.3451, 0.2745, 0.3333,  ..., 0.4745, 0.4588, 0.4588],
        [0.3882, 0.3529, 0.4196,  ..., 0.4941, 0.4941, 0.4784],
        [0.4549, 0.3804, 0.4745,  ..., 0.5098, 0.5176, 0.4863],
        ...,
        [0.5412, 0.3686, 0.1922,  ..., 0.1373, 0.1451, 0.1569],
        [0.5765, 0.2941, 0.1882,  ..., 0.1216, 0.1412, 0.1804],
        [0.5765, 0.2941, 0.1882,  ..., 0.1294, 0.1412, 0.1804]],

       [[0.2392, 0.1569, 0.2039,  ..., 0.4627, 0.4471, 0.4471],
        [0.2627, 0.2235, 0.2745,  ..., 0.4824, 0.4824, 0.4667],
        [0.3020, 0.2275, 0.3098,  ..., 0.4980, 0.5059, 0.4745],
        ..

In [6]:
from torch import nn

model = nn.Sequential(
                            nn.Conv2d(in_channels=3,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                            nn.ReLU(),
                            nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2),
                            nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                            nn.ReLU(),
                            nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2),
                            nn.Flatten(),
                            nn.Linear(in_features=28090,out_features=1) 
)

In [7]:
model

Sequential(
  (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (1): ReLU()
  (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (6): ReLU()
  (7): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=28090, out_features=1, bias=True)
)

In [8]:
from torchinfo import summary

summary(model,
        input_size=(1,3,224,224),
        verbose=1,
        col_names=["input_size","output_size","num_params","trainable"],
        row_settings=["var_names"],
        col_width=18);

Layer (type (var_name))                  Input Shape        Output Shape       Param #            Trainable
Sequential (Sequential)                  [1, 3, 224, 224]   [1, 1]             --                 True
├─Conv2d (0)                             [1, 3, 224, 224]   [1, 10, 222, 222]  280                True
├─ReLU (1)                               [1, 10, 222, 222]  [1, 10, 222, 222]  --                 --
├─Conv2d (2)                             [1, 10, 222, 222]  [1, 10, 220, 220]  910                True
├─ReLU (3)                               [1, 10, 220, 220]  [1, 10, 220, 220]  --                 --
├─MaxPool2d (4)                          [1, 10, 220, 220]  [1, 10, 110, 110]  --                 --
├─Conv2d (5)                             [1, 10, 110, 110]  [1, 10, 108, 108]  910                True
├─ReLU (6)                               [1, 10, 108, 108]  [1, 10, 108, 108]  --                 --
├─Conv2d (7)                             [1, 10, 108, 108]  [1, 10, 106, 106

In [9]:
loss_fn = th.nn.BCEWithLogitsLoss()

optimizer = th.optim.Adam(model.parameters(),
                          lr=0.001)

In [10]:
def accuracy(y_preds,y_true):
    correct = th.eq(y_preds,y_true).sum().item()
    acc = (correct/len(y_preds)) * 100
    return acc

In [11]:
EPOCHS = 30
TRAIN_DL_LEN = len(train_data_loader)
TEST_DL_LEN = len(test_data_loader)

model.to(DEVICE)
for epoch in range(EPOCHS):
    start = time.time() 
    model.train()
    epoch_train_loss = 0.0
    epoch_train_accuracy = 0.0
    for X,Y in train_data_loader:
        X = X.to(DEVICE)
        Y = Y.type(th.float32).to(DEVICE)
        
        y_logits = model(X).squeeze()
        y_preds = th.round(th.sigmoid(y_logits))
        
        loss = loss_fn(y_logits,Y)
        acc = accuracy(y_preds,Y)
        
        epoch_train_loss += loss
        epoch_train_accuracy += acc
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    model.eval()
    epoch_test_loss = 0.0
    epoch_test_accuracy = 0.0
    with th.no_grad():
        for X,Y in test_data_loader:
            X = X.to(DEVICE)
            Y = Y.type(th.float32).to(DEVICE)
            
            y_logits = model(X).squeeze()
            y_preds = th.round(th.sigmoid(y_logits))
            
            loss = loss_fn(y_logits,Y)
            acc = accuracy(y_preds,Y)
            
            epoch_test_loss += loss
            epoch_test_accuracy += acc
    
    epoch_train_loss /= TRAIN_DL_LEN
    epoch_train_accuracy /= TRAIN_DL_LEN
    epoch_test_loss /= TEST_DL_LEN
    epoch_test_accuracy /= TEST_DL_LEN
    end = time.time()
    print(f"Epoch {epoch}[{end-start}] :- |Train Loss:- {epoch_train_loss:.2f} <--------> Train Accuracy:- {epoch_train_accuracy:.2f}|++++++++|Test Loss:- {epoch_test_loss:.2f} <--------> Test Accuracy :- {epoch_test_accuracy:.2f}|")

Epoch 0[5.885272979736328] :- |Train Loss:- 0.70 <--------> Train Accuracy:- 51.27|++++++++|Test Loss:- 0.70 <--------> Test Accuracy :- 48.29|
Epoch 1[5.657883405685425] :- |Train Loss:- 0.69 <--------> Train Accuracy:- 54.05|++++++++|Test Loss:- 0.66 <--------> Test Accuracy :- 61.68|
Epoch 2[5.598556756973267] :- |Train Loss:- 0.68 <--------> Train Accuracy:- 59.26|++++++++|Test Loss:- 0.65 <--------> Test Accuracy :- 60.04|
Epoch 3[5.5212342739105225] :- |Train Loss:- 0.68 <--------> Train Accuracy:- 58.10|++++++++|Test Loss:- 0.69 <--------> Test Accuracy :- 55.65|
Epoch 4[5.5367302894592285] :- |Train Loss:- 0.67 <--------> Train Accuracy:- 58.91|++++++++|Test Loss:- 0.63 <--------> Test Accuracy :- 62.13|
Epoch 5[5.565121173858643] :- |Train Loss:- 0.66 <--------> Train Accuracy:- 59.95|++++++++|Test Loss:- 0.65 <--------> Test Accuracy :- 63.10|
Epoch 6[5.705610752105713] :- |Train Loss:- 0.65 <--------> Train Accuracy:- 63.89|++++++++|Test Loss:- 0.62 <--------> Test Accuracy 